In [110]:
import torch

In [111]:
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [112]:
project_name='assign 1' 

In [113]:
DATASET_URL = "https://gist.github.com/BirajCoder/5f068dfe759c1ea6bdfce9535acdb72d/raw/c84d84e3c80f93be67f6c069cbdc0195ec36acbd/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: .\insurance.csv


In [114]:
data=pd.read_csv(DATA_FILENAME)
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [115]:
name="anurag"

In [116]:
def custom(data, rand_str):
    data_copy=data.copy(deep=True)
    data_copy=data_copy.sample(int(0.95*len(data_copy)), random_state=int(ord(rand_str[0])))
    data_copy.bmi=data_copy.bmi*ord(rand_str[1])/100.
    data_copy.charges=data_copy.charges*ord(rand_str[2])/100.
    if(ord(rand_str[3])%2==1):
        data_copy=data_copy.drop(['region'], axis=1)
    return data_copy

In [117]:
data_copy=custom(data, name)
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [118]:
categorical_cols=[]
for col in data.columns:
    if  not pd.api.types.is_numeric_dtype(data[col]) or pd.api.types.is_categorical_dtype(data[col]):
        categorical_cols.append(col)
print(categorical_cols)

['sex', 'smoker', 'region']


In [119]:
input_cols=[]
for col in data:
    if col!='charges':
        input_cols.append(col)
print(input_cols)

['age', 'sex', 'bmi', 'children', 'smoker', 'region']


In [120]:
output_cols=[]
for col in data:
    if col=='charges':
        output_cols.append(col)
print(output_cols)

['charges']


In [121]:
def tonumpy(dataframe):
    data1=dataframe.copy(deep=True)
    for col in categorical_cols:
        data1[col]=data1[col].astype('category').cat.codes
    inputs_array=data1[input_cols].to_numpy()
    outputs_array=data1[output_cols].to_numpy()
    return inputs_array, outputs_array

In [122]:
input, target=tonumpy(data)

In [123]:
input.dtype

dtype('float64')

In [124]:
from torch.utils.data import TensorDataset

In [125]:
inputs=torch.tensor(input)
targets=torch.tensor(target)

In [126]:
dataset=TensorDataset(inputs, targets)

In [127]:
n_rows, n_cols= data.shape

In [128]:
val_size=int(0.8*len(data))
train_size=len(dataset)-val_size

In [129]:
from torch.utils.data import random_split

In [130]:
train_dataset, val_dataset=random_split(dataset, [train_size, val_size])

In [131]:
train_loader=DataLoader(train_dataset, batch_size=64, shuffle=True )
val_loader=DataLoader(val_dataset, batch_size=64, shuffle=True )

In [132]:
for x,y in train_loader:
    print("inputs:", x)
    print("outputs:", y)
    break

inputs: tensor([[61.0000,  0.0000, 44.0000,  0.0000,  0.0000,  3.0000],
        [19.0000,  1.0000, 34.8000,  0.0000,  1.0000,  3.0000],
        [27.0000,  1.0000, 30.3000,  3.0000,  0.0000,  3.0000],
        [30.0000,  1.0000, 31.4000,  1.0000,  0.0000,  3.0000],
        [44.0000,  1.0000, 22.1350,  2.0000,  0.0000,  0.0000],
        [19.0000,  1.0000, 27.2650,  2.0000,  0.0000,  1.0000],
        [26.0000,  0.0000, 28.7850,  0.0000,  0.0000,  0.0000],
        [19.0000,  0.0000, 30.0200,  0.0000,  1.0000,  1.0000],
        [42.0000,  0.0000, 26.1800,  1.0000,  0.0000,  2.0000],
        [18.0000,  0.0000, 39.1600,  0.0000,  0.0000,  2.0000],
        [40.0000,  0.0000, 22.2200,  2.0000,  1.0000,  2.0000],
        [50.0000,  1.0000, 37.0700,  1.0000,  0.0000,  2.0000],
        [29.0000,  1.0000, 34.4000,  0.0000,  1.0000,  3.0000],
        [35.0000,  1.0000, 17.8600,  1.0000,  0.0000,  1.0000],
        [32.0000,  0.0000, 24.6000,  0.0000,  1.0000,  3.0000],
        [26.0000,  1.0000, 32.49

In [133]:
input_size = len(input_cols)
output_size = len(output_cols)

In [134]:
import torch.nn as nn

In [135]:
import torch
import torch.nn as nn

class InsuranceModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)  # Fill in the input and output sizes

    def forward(self, xb):
        xb = xb.float()  # Assuming you want to work with float data type
        out = self.linear(xb)
        return out


    def training_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Ensure consistent data type (e.g., float)
        targets = targets.float()
    # Calculate loss (for example, mean squared error)
        loss = nn.MSELoss()(out, targets)
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = nn.MSELoss()(out,targets)                     # fill this
        return {'val_loss': loss.detach()}
    def validation_epoch_end(self, outputs):
        batch_losses = [torch.tensor(x['val_loss']) for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}

    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))


In [136]:
model=InsuranceModel(input_size, output_size)

In [137]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0128,  0.0348,  0.4074, -0.2988,  0.0056, -0.2565]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.1346], requires_grad=True)]

In [138]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

In [139]:
print(evaluate(model, val_loader))

{'val_loss': 328972428.2500011}


In [140]:
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
epochs_list = [20, 30, 40, 50, 60]  # You can adjust the number of epochs as needed


In [141]:
for lr in learning_rates:
    for epochs in epochs_list:
        # Create a new instance of your model (if needed) for each training run
        model = InsuranceModel(input_size, output_size)
        
        # Train the model using the current learning rate and number of epochs
        history = fit(epochs, lr, model, train_loader, val_loader)
        
        # Print or store the training results (e.g., loss, accuracy) for analysis
        print(f"Learning Rate: {lr}, Epochs: {epochs}")
        # You can print or analyze the history variable to get training results
        
        # Optionally, you can save the trained model weights for each run if needed
        torch.save(model.state_dict(), f"model_lr_{lr}_epochs_{epochs}.pt")


Epoch [20], val_loss: nan
Learning Rate: 0.01, Epochs: 20
Epoch [20], val_loss: nan
Epoch [30], val_loss: nan
Learning Rate: 0.01, Epochs: 30
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Learning Rate: 0.01, Epochs: 40
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [50], val_loss: nan
Learning Rate: 0.01, Epochs: 50
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Learning Rate: 0.01, Epochs: 60
Epoch [20], val_loss: nan
Learning Rate: 0.001, Epochs: 20
Epoch [20], val_loss: nan
Epoch [30], val_loss: nan
Learning Rate: 0.001, Epochs: 30
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Learning Rate: 0.001, Epochs: 40
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [50], val_loss: nan
Learning Rate: 0.001, Epochs: 50
Epoch [20], val_loss: nan
Epoch [40], val_loss: nan
Epoch [60], val_loss: nan
Learning Rate: 0.001, Epochs: 60
Epoch [20], val_loss: 133644293.3585
Learning Rate: 0.0001, Epochs: 20
Epoch [20], val_loss: 131317497.

In [142]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)               # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [145]:
input, target = val_dataset[10]
predict_single(input, target, model)

Input: tensor([62.0000,  1.0000, 38.8300,  0.0000,  0.0000,  2.0000],
       dtype=torch.float64)
Target: tensor([12981.3457], dtype=torch.float64)
Prediction: tensor([14598.7480])
